<a href="https://colab.research.google.com/github/HausReport/ClubRaiders/blob/master/notebooks/ExpandRetreat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import math
import pandas as pd
import plotly.graph_objects as go
import humanize
import datetime as dt

In [35]:
import sys

!{sys.executable} -m pip install --force-reinstall --upgrade "git+https://github.com/HausReport/ClubRaiders.git#egg=craid"
import craid.eddb.loader.DataProducer as dp
import pandas as pd
import logging
import plotly.graph_objs as go
from numpy import *


logging.getLogger().addHandler(logging.StreamHandler())
logging.getLogger().level = logging.DEBUG

arrays = dp.getDataArrays()
df: pd.DataFrame = arrays['dataFrame']

  Cloning https://github.com/HausReport/ClubRaiders.git to /tmp/pip-install-qcz2bqoi/craid
  Running command git clone -q https://github.com/HausReport/ClubRaiders.git /tmp/pip-install-qcz2bqoi/craid
     |████████████████████████████████| 81kB 2.3MB/s 
     |████████████████████████████████| 3.4MB 8.2MB/s 
     |████████████████████████████████| 194kB 42.5MB/s 
     |████████████████████████████████| 1.8MB 41.9MB/s 
     |████████████████████████████████| 1.0MB 42.2MB/s 
     |████████████████████████████████| 102kB 11.1MB/s 
     |████████████████████████████████| 9.5MB 48.4MB/s 
     |████████████████████████████████| 71kB 9.0MB/s 
     |████████████████████████████████| 788kB 49.1MB/s 
     |████████████████████████████████| 14.5MB 308kB/s 
     |████████████████████████████████| 235kB 50.5MB/s 
     |████████████████████████████████| 184kB 50.7MB/s 
     |████████████████████████████████| 133kB 50.1MB/s 
     |████████████████████████████████| 460kB 46.3MB/s 
     |███████████████

Acquired lock.
Loading from AWS
Memory tracker: 0:200,593,408
1- downloading to: /tmp/smol-factions.jsonl.gz
2 - downloading [https://erlaed.s3.us-east-2.amazonaws.com/smol-factions.jsonl.gz] to [/tmp/smol-factions.jsonl.gz] data file.
Starting new HTTPS connection (1): erlaed.s3.us-east-2.amazonaws.com:443
https://erlaed.s3.us-east-2.amazonaws.com:443 "GET /smol-factions.jsonl.gz HTTP/1.1" 200 14856
Found data file: /tmp/smol-factions.jsonl.gz
Read 628 lines of faction data
Memory tracker: 0.5:201,732,096
1- downloading to: /tmp/smol-systems_populated.jsonl.gz
2 - downloading [https://erlaed.s3.us-east-2.amazonaws.com/smol-systems_populated.jsonl.gz] to [/tmp/smol-systems_populated.jsonl.gz] data file.
Starting new HTTPS connection (1): erlaed.s3.us-east-2.amazonaws.com:443
https://erlaed.s3.us-east-2.amazonaws.com:443 "GET /smol-systems_populated.jsonl.gz HTTP/1.1" 200 24453
Found data file: /tmp/smol-systems_populated.jsonl.gz
Read 166 lines of systems data
Memory tracker: 1:202,375

In [20]:
def expandOneDay(inf: float, pop: int):
  top = (36 - math.log(pop,2))
  bottom = (100+ (36 - math.log(pop,2)))
  ret = 100.0 * (inf + top)/ bottom
  if ret> 100.0:
     ret = 100.0
  if ret<1.0:
     ret = 1.0
  return ret

def dailyIncrease(inf: float, pop: int):
  return expandOneDay(inf,pop) - inf

In [37]:
from craid.eddb.faction.Strategy import Strategy
from craid.eddb.faction.Simulation import Simulation

In [39]:
strat = Strategy()
theSim = Simulation(strat)

In [21]:
def retreatOneDay(inf: float, pop: int):
  top = (36 - math.log(pop,2))
  bottom = (100+ (36 - math.log(pop,2)))
  ret = inf - ((100 * ( (100-inf) + top)/ bottom) - (100-inf))
  if ret> 100.0:
     ret = 100.0
  if ret<1.0:
     ret = 1.0
  return ret

def dailyDecrease(inf: float, pop: int):
  return expandOneDay(inf,pop) - inf

In [41]:
strat.retreatOneDay(9.2,2200)

7.3660866155447415

In [42]:
strat.expandOneDay(65.9,2200)

72.6974398271657

In [24]:
def daysToPendingExpansion(inf: float, pop: int):
  tinf = inf
  day = 0
  while tinf < 75.0:
    day += 1
    tinf = expandOneDay(tinf, pop)
  return day

def daysToPendingRetreat(inf: float, pop: int):
  tinf = inf
  day = 0
  while tinf > 2.5:
    day += 1
    tinf = retreatOneDay(tinf, pop)
  return day

def daysToExpansion(inf: float, pop: int):
  ret = daysToPendingExpansion(inf,pop)
  ret += 5 # pending days
  ret += 7 # active expansion https://discordapp.com/channels/483005833853009950/483005833853009952/759126832750002186
  return ret

def daysToRetreat(inf: float, pop: int):
  ret = daysToPendingRetreat(inf,pop)
  ret += 1 # pending day
  ret += 6 # active retreat
  ret += 1
  return ret

In [44]:
theSim.daysToPendingExpansion(65.9,2200)

2

In [45]:
theSim.daysToPendingRetreat(65.9,2200)

15

In [46]:
humanize.naturaldelta(dt.datetime.now() + dt.timedelta(days=0.5 ))

'11 hours'

In [28]:

def getRetreatFrame(inf: float, pop: int):
  dayNum = []
  infs = []
  day = 0

  dayNum.append(day)
  infs.append(inf)

  tinf = inf
  while tinf > 2.5:
    day +=1
    tinf = retreatOneDay(tinf,pop)
    dayNum.append(day)
    infs.append(tinf)

  n = 0
  while n<=8:
    day +=1
    tinf = retreatOneDay(tinf,pop)
    dayNum.append(day)
    infs.append(tinf)
    n += 1

  df = pd.DataFrame( list(zip(dayNum, infs)), columns=["dayNum","inf"])
  return df

def getAdvanceFrame(inf: float, pop: int, nDays: int):
  dayNum = []
  infs = []
  day = 0

  dayNum.append(day)
  infs.append(inf)

  tinf = inf
  while day<= nDays:
    day +=1
    tinf = expandOneDay(tinf,pop)
    dayNum.append(day)
    infs.append(tinf)

  df = pd.DataFrame( list(zip(dayNum, infs)), columns=["dayNum","inf"])
  return df

In [29]:
inf = 36.0
pop = 36000

nDays = daysToRetreat(inf, pop)
df = getRetreatFrame(inf,pop)
df2 = getAdvanceFrame(inf, pop, nDays)

fig = go.Figure()
fig.add_trace(go.Scatter(x=df2.dayNum, y=df2.inf,
                    mode='lines',
                    name='Ally',
                    marker_color="#0000ff"))
fig.add_trace(go.Scatter(x=df.dayNum, y=df.inf,
                    mode='lines',
                    name='Target',
                    marker_color="#ff0000"))
retLine = list( 2.5 for i in range(df.dayNum.size))
expLine = list( 75.0 for i in range(df.dayNum.size))
fig.add_trace(go.Scatter(x=df2.dayNum, y=retLine,
                    mode='lines',
                    name='Retreat',
                    marker_color="#ff0000",
                    line_dash="dot"))
fig.add_trace(go.Scatter(x=df2.dayNum, y=expLine,
                    mode='lines',
                    name='Expand',
                    marker_color="#00ff00",
                    line_dash="dot"))
fig.show()



In [30]:
def getSimulationFrame(ally_inf: float, targ_inf: float, pop: int):
  #nDays = daysToRetreat(inf, pop)

  dayNum = []
  ally_infs = []
  targ_infs = []
  ally_note = []
  targ_note = []

  day = 0

  dayNum.append(day)
  targ_infs.append(targ_inf)
  ally_infs.append(ally_inf)
  targ_note.append("")
  ally_note.append("")

  tinf = targ_inf
  ainf = ally_inf

  expansion_day = 0
  retreat_day = 0

  while retreat_day<= 7:
    day +=1
    # Save current day's numbers
    painf = ainf
    ptinf = tinf

    # Get next day's numbers
    ainf = expandOneDay(ainf,pop)
    tinf = retreatOneDay(tinf,pop)

    #detect crossing
    if (  (painf<ptinf) and (ainf>=tinf) ):
      minf = (ainf + tinf) / 2.0
      for i in range(5):
          dayNum.append(day)
          targ_infs.append(minf)
          ally_infs.append(minf)
          note = ""
          if i == 0:
            note = "Pending Conflict"
          else:
            note = "Conflict day " + str(i)
          targ_note.append(note)
          ally_note.append(note)
          day += 1
    else: 
      dayNum.append(day)
      note = ""
      if tinf<2.5:
        if retreat_day == 0:
          note = "Pending Retreat"
        else:
          note = "Retreat day " + str(retreat_day)
        retreat_day += 1

      targ_infs.append(tinf)
      targ_note.append(note)

      note = ""
      if ainf>75.0:
        if expansion_day < 5:
          note = "Pending Expansion day " + str(expansion_day+1)
        elif expansion_day<12:
          note = "Expansion day " + str(expansion_day-5+1)
          if expansion_day>=10:
            note = note + " *"
        elif expansion_day==12:
          note = "Expansion *"
          ainf = ainf - 10.0 #expansion tax
        else:
          note=""
        expansion_day += 1
      ally_infs.append(ainf)
      ally_note.append(note)

  #df = pd.DataFrame( list(zip(dayNum, ally_infs, targ_infs)), columns=["dayNum","ally_inf","target_inf"])
  df = pd.DataFrame( list(zip(dayNum, ally_infs, ally_note, targ_infs, targ_note)), columns=["dayNum","ally_inf","ally_note","target_inf","targ_note"])
  return df

def getDatedSimulationFrame(startDate: dt.datetime, ally_inf: float, targ_inf: float, pop: int):
    df = getSimulationFrame(ally_inf, targ_inf, pop)
    dates = []
    for i in range(df.dayNum.size):
      theDate: dt.date = (startDate + dt.timedelta(days=i)).date()
      dates.append(theDate)

    df['date'] = dates
    return df

In [31]:
sim = getDatedSimulationFrame( dt.datetime.now(), 30, 40, 40000)

In [33]:
def getSimulationFigure( sim: pd.DataFrame ):
  ally_color="pink"
  target_color = "red"
  conflict_color = "yellow"
  expansion_color = "magenta"
  retreat_color ="deeppink"
  
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=sim.date, y=sim.ally_inf,
                    mode='lines',
                    name='Ally',
                    marker_color="blue"))
  fig.add_trace(go.Scatter(x=sim.date, y=sim.target_inf,
                    mode='lines',
                    name='Target',
                    marker_color="red"))
  retLine = list( 2.5 for i in range(sim.dayNum.size))
  expLine = list( 75.0 for i in range(sim.dayNum.size))
  fig.add_trace(go.Scatter(x=sim.date, y=retLine,
                    mode='lines',
                    name='Retreat',
                    marker_color="#ff0000",
                    line_dash="dot"))
  fig.add_trace(go.Scatter(x=sim.date, y=expLine,
                    mode='lines',
                    name='Expand',
                    marker_color="#00ff00",
                    line_dash="dot"))
  #fig.update_yaxes(type="log")
  return fig

fig = getSimulationFigure(sim)
fig.show()

In [34]:
sim

,dayNum,ally_inf,ally_note,target_inf,targ_note,date
0,0,30.000000,,40.000000,,2020-10-20
1,1,37.573759,Pending Conflict,37.573759,Pending Conflict,2020-10-21
2,2,37.573759,Conflict day 1,37.573759,Conflict day 1,2020-10-22
3,3,37.573759,Conflict day 2,37.573759,Conflict day 2,2020-10-23
4,4,37.573759,Conflict day 3,37.573759,Conflict day 3,2020-10-24
5,5,37.573759,Conflict day 4,37.573759,Conflict day 4,2020-10-25
6,7,51.960876,,27.450928,,2020-10-26
7,8,60.203617,,22.740790,,2020-10-27
8,9,67.032037,,18.838836,,2020-10-28
9,10,72.688809,,15.606395,,2020-10-29
